In [ ]:
import base64
import os
import logging
import time
import jwt
import requests
from urllib.parse import urlencode
from typing import Optional

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)



githubapp_id:str = "..."
githubapp_installation_id:str = "..."
githubapp_pem_key:str = "..."
repo:str = '...'
organization:str = '...'
branch='main'
branch_dev='main'
pr_number = '5'
basic = False

proxies = {}


def __get_github_token() -> str:
    with open(githubapp_pem_key, 'rb') as pem_file:
        signing_key = pem_file.read()
    payload = {
        # Issued at time
        'iat': int(time.time()),
        # JWT expiration time (10 minutes maximum)
        'exp': int(time.time()) + 10,
        # GitHub App's client ID
        'iss': githubapp_id
    }
    # Create JWT
    jwt_token = jwt.encode(payload, signing_key, algorithm='RS256')
    # Get the installation access token based on the JWT
    headers:dict = {
        "Authorization": f"Bearer {jwt_token}",
        "Accept": "application/vnd.github+json",
        "X-GitHub-Api-Version": "2022-11-28"
    }
    url:str = f"https://api.github.com/app/installations/{githubapp_installation_id}/access_tokens"
    
    response:requests.Response = __call_github_api(url=url, headers=headers, method="POST")
    token = response.json().get("token")
    if token is None:
        raise ValueError("Failed to retrieve GitHub token. Response: {}".format(response.json()))
    
    #print("token:" + token)
    return token

def __github_headers() -> dict:
    bearer_token:str = __get_github_token()
    headers:dict = {
        "Authorization": f"Bearer {bearer_token}",
        "Accept": "application/vnd.github+json",
        "X-GitHub-Api-Version": "2022-11-28"
    }
    return headers

def __call_github_api(url:str, headers:dict = None, body:dict = None, method:str = "GET") -> requests.Response:
    try:
        return requests.request(method, url, headers=headers or __github_headers(), proxies=proxies, verify=False, json=body)
    except Exception as e:
        logging.exception(f"Falha ao chamar a API do Github: {url}")
        return {}

def __fetch_pr_info(organization:str, repo:str, pr_number:str, info:str = None) -> dict:
    url = f"https://api.github.com/repos/{organization}/{repo}/pulls/{pr_number}"
    if info:
        url += f"/{info}"
    return __call_github_api(url)

def __load_prom_files(directory) -> list[str]:
    prom_files = []
    try:
        for filename in os.listdir(directory):
            if filename.endswith(".prom"):
                with open(os.path.join(directory, filename), 'r') as file:
                    prom_files.append(file.read())
    except FileNotFoundError:
        print(f"Diretório {directory} não encontrado.")
    return prom_files

def __call_openia_http(system_prompt,user_prompt):
    
  # Configuration
  API_KEY = "3e814a46cd264b108f105f5ea739dddf"
  headers = {
      "Content-Type": "application/json",
      "api-key": API_KEY,
  }

  # Payload for the request
  payload = {
    "messages": [
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_prompt
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_prompt
          }
        ]
      }
    ],
    "temperature": 0.7,
    "top_p": 0.95,
    "max_tokens": 4000
  }

  ENDPOINT = "https://labws03.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"

  # Send request
  try:
      response = requests.post(ENDPOINT, headers=headers, json=payload)
      response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
  except requests.RequestException as e:
      raise SystemExit(f"Failed to make the request. Error: {e}")

  # Handle the response as needed (e.g., print or process)
  response_json = response.json()
  text = response_json["choices"][0]["message"]["content"]

  # print(text)

  return text

def __find_best_prompt(user_request, prompts):
    best_prompt = None
    best_score = float('-inf')

    for prompt in prompts:

            user_request = f"Qual prompt é mais adequado para a seguinte solicitação: '{user_request}'? Aqui está o prompt: '{prompt}'."
            system_prompt = f"Responda apenas com um número de 0 até 10"
            response = __call_openia_http(system_prompt,user_request)

            try:
                score = float(response)
            except ValueError :
                score = 0 

            if score > best_score:  
                best_score = score
                best_prompt = prompt


    return best_prompt

def __fetch_pr_diff(organization:str, repo:str, pr_number:str) -> dict:
    headers:dict = __github_headers()
    headers['Accept'] = "application/vnd.github.diff"
    url = f"https://api.github.com/repos/{organization}/{repo}/pulls/{pr_number}"    
    return __call_github_api(url, headers)

def __create_pr_comment(organization:str,repo:str, pr_number:int, comment:str) -> dict:
    body:dict = {
        "body": comment,
    }
    url = f"https://api.github.com/repos/{organization}/{repo}/issues/{pr_number}/comments"
    return __call_github_api(url, body=body, method="POST")

def __create_pr_comment_id(organization:str,repo:str, pr_number:int, commit_id:str, file:str, comment:str) -> dict:
    body:dict = {
        "body": comment,
        "commit_id": commit_id,
        "path": file,
        "subject_type": "file"
    }
    url = f"https://api.github.com/repos/{organization}/{repo}/pulls/{pr_number}/comments"
    return __call_github_api(url, body=body, method="POST")

def __fetch_branch_info(organization:str, repo:str, branch:str) -> requests.Response:
    url = f"https://api.github.com/repos/{organization}/{repo}/branches/{branch}"
    return __call_github_api(url)

def __create_new_branch(organization:str, repo:str, branch:str, base_sha_commit:str = None) -> requests.Response:
    branch_info:requests.Response = __fetch_branch_info(organization, repo, branch)
    if branch_info.ok:
        logging.info(f"Branch '{branch}' já existe.")
        return branch_info

    if not base_sha_commit:
        develop = __fetch_branch_info(organization, repo, branch_dev).json()
        base_sha_commit = develop["commit"]["sha"]

    url = f"https://api.github.com/repos/{organization}/{repo}/git/refs"
    body = {
        "ref": f"refs/heads/{branch}",
        "sha": base_sha_commit
    }
    logging.info(f"Criando branch '{branch}'.")
    return __call_github_api(url, body=body, method="POST")

def __create_new_commit(organization:str, repo:str, newbranch:str, commit_message:str, file_path:str, file_content:str) -> requests.Response:
    url = f"https://api.github.com/repos/{organization}/{repo}/contents/{file_path}"
    body = {
        "message": commit_message,
        "content": base64.b64encode(file_content.encode("utf-8")).decode("utf-8"),
        "branch": newbranch
    }

    # Verifica se o arquivo ja existe
    file:requests.Response = __call_github_api(url, body=body)
    if file.ok:
        sha = file.json()["sha"]
        body.update({"sha": sha})

    #print(file) 
    # Cria a branch caso nao exista
    response_branch =__create_new_branch(organization, repo, newbranch)

    logging.info(f"Realizando commit do arguivo '{file_path}' no branch '{newbranch}'.")
    response:requests.Response = __call_github_api(url, body=body, method="PUT")
    logging.info(response.json())
    return response

def __create_pull_request(organization:str, repo:str, pr_title:str, pr_description:str, head_brach:str, base_branch:str) -> requests.Response:
    url = f"https://api.github.com/repos/{organization}/{repo}/pulls"
    body = {
        "title": pr_title,
        "body": pr_description,
        "head": head_brach,
        "base": base_branch
    }
    return __call_github_api(url, body=body, method="POST")


pr_info = __fetch_pr_info(organization,repo,pr_number).json()
pr_intetion = pr_info['body']
directory_path:str = "C:\\projetos\\IANotebook\\prompts"
prom_files_content = __load_prom_files(directory_path)
best_prompt = __find_best_prompt(pr_intetion, prom_files_content)

# Obtém as informações dos commits do pull request
pr_commits:list[dict] = __fetch_pr_info(organization=organization, repo=repo, pr_number=pr_number, info='commits').json()
logging.info("Commits:")
logging.info(pr_commits)

# Obtém o diff do pull request
pr_diff:str = __fetch_pr_diff(organization=organization, repo=repo, pr_number=pr_number).text
logging.info("Diff: ")
logging.info(pr_diff)

# Se a variável 'basic' estiver definida como True, realiza uma análise básica
if basic:
    print("Cria um comentário no PR com a resposta obtida")
    # Cria a solicitação do usuário combinando o melhor prompt e o diff do PR
    user_request = best_prompt + ". Segue o Diff do PR para analise:" + pr_diff
    # Define o prompt do sistema para separar as análises por bloco de arquivo modificado
    system_prompt = "Separe as analises procurando por 'diff --git', cada bloco representa um arquivo modificado. Para cada bloco, crie uma analise independente."
    # Chama a API do OpenAI para obter a resposta
    response = __call_openia_http(system_prompt, user_request)
    # Cria um comentário no PR com a resposta obtida
    __create_pr_comment(organization, repo, pr_number, response)

# Caso contrário, realiza uma análise detalhada por commit
else:
    print("realiza uma análise detalhada por commit")
    
    # Obtém o timestamp atual em segundos
    timestamp = int(time.time())  
    # Cria uma nova branch para a refatoração
    newBranch = "code-review-pr" + pr_number + "-" + str(timestamp)

    for commit in pr_commits:
        commit_id = commit.get('sha')
        # Obtém os arquivos modificados no commit
        commit_files = __call_github_api(f"https://api.github.com/repos/wilsonsantosnet/githubapp/commits/{commit_id}").json()
        for file in commit_files['files']:
            filename = file['filename']
            patch = file['patch']
            download_url = file['raw_url']
            print(f"Nome do arquivo: {filename}")

            # Cria a solicitação do usuário combinando o melhor prompt e o patch do arquivo
            user_request = best_prompt + ". Segue o Diff do PR para analise:" + patch
            system_prompt = ""
            # Chama a API do OpenAI para obter a resposta
            response = __call_openia_http(system_prompt, user_request)
            # Cria um comentário no PR com a resposta obtida, incluindo o ID do commit e o nome do arquivo
            __create_pr_comment_id(organization, repo, pr_number, commit_id, filename, response)

            # Obtém o conteúdo bruto do arquivo
            raw_file = __call_github_api(download_url).text

            # Cria a solicitação do usuário para refatorar o código com base nas sugestões fornecidas
            user_request = "Com base nas sugestões fornecidas pela sua analise: {response} poderia refatorar esse código:" + raw_file 
            system_prompt = "Retorne apenas código refatorado"
            response_refact = __call_openia_http(system_prompt, user_request)

            
            # Cria um novo commit na nova branch com a refatoração do arquivo
            newCommit = __create_new_commit(organization, repo, newBranch, "Refatoração do arquivo " + filename, filename, response_refact)

            # Cria um pull request com a refatoração
            new_pr_title = "Refatoração do pr " + pr_number + "-" + str(timestamp)
            __create_pull_request(organization, repo, new_pr_title , new_pr_title, newBranch, branch_dev)


realiza uma análise detalhada por commit
Nome do arquivo: nodejs-example/index4.js
Nome do arquivo: nodejs-example/index5.js
